In [2]:
import numpy as np
import torch

from tqdm.auto import tqdm
from pathlib import Path

import itertools

import os
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv(), verbose=True)

True

In [3]:
!rm -r /home/huze/.cache/checkpoints

In [4]:
exp_configs = [p for p in Path('../../src/config/experiments/algonauts2021/').iterdir() if p.name.endswith('.yml')]

In [5]:
exp_configs

[PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_segswin.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_simclr.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_pyconvsegnet.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_mobyswin.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_bdcnvgg.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_3d_resnet.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_densnet.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_colorizer.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_audio_vgg.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_3d_flow.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_3d_swin.yml')]

In [6]:
os.environ['RESULTS_DIR']

'/data/huze/ray_results/algonauts2021/'

In [7]:
from src.grid_runner import run_single_train
from src.grid_runner import run_single_tune_config
from src.config import get_cfg_defaults, combine_cfgs
from ray import tune

In [8]:
debug = False

In [9]:
# anatomical rois

analysis_list = []
for exp_config in exp_configs:
    cfg = combine_cfgs(
        path_cfg_data=exp_config,
        list_cfg_override=['DEBUG', debug]
    )
    
    name = exp_config.name.replace('.yml', '')

    # multi-layer 1 run for each ROI
    analysis = tune.run(
        tune.with_parameters(
            run_single_tune_config,
            cfg=cfg
        ),
        config={
            'DATASET.ROI': tune.grid_search(['LOC', 'FFA', 'STS', 'EBA', 'PPA', 'V1', 'V2', 'V3', 'V4', 'REST']),
            'TRAINER.CALLBACKS.BACKBONE.DEFROST_SCORE': tune.grid_search([1.]),
        },
        local_dir=cfg.RESULTS_DIR,
        resources_per_trial={"cpu": 4, "gpu": 1},
        name=name + '_multilayer_freeze_anatomicalROI',
        verbose=1,
        resume='AUTO',
    )
    analysis_list.append(analysis)

2022-03-19 02:37:25,809	INFO tune.py:636 -- Total run time: 0.13 seconds (0.00 seconds for the tuning loop).


In [10]:
# my rois

analysis_list = []
for exp_config in exp_configs:
    cfg = combine_cfgs(
        path_cfg_data=exp_config,
        list_cfg_override=['DEBUG', debug]
    )
    
    name = exp_config.name.replace('.yml', '')

    # multi-layer 1 run for each ROI
    analysis = tune.run(
        tune.with_parameters(
            run_single_tune_config,
            cfg=cfg
        ),
        config={
            'DATASET.ROI': tune.grid_search([f'S-htROI{i+1}' for i in range(8)]),
            'TRAINER.CALLBACKS.BACKBONE.DEFROST_SCORE': tune.grid_search([1.]),
        },
        local_dir=cfg.RESULTS_DIR,
        resources_per_trial={"cpu": 4, "gpu": 1},
        name=name + '_multilayer_freeze_htROI',
        verbose=1,
        resume='AUTO',
    )
    analysis_list.append(analysis)

2022-03-19 09:02:43,840	INFO tune.py:636 -- Total run time: 2560.26 seconds (2560.14 seconds for the tuning loop).


In [38]:
rm /home/huze/Algonauts_2021_data/voxel_indexs/*htROI*

E0322 02:24:53.737594223  273619 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [39]:
!cp tmp/notebook012/*htROI* /home/huze/Algonauts_2021_data/voxel_indexs/

E0322 02:24:55.248728911  273619 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [24]:
# ablation htROI

rois = ['XS-htROI4', 'XS-htROI5', 'XS-htROI6', 'XS-htROI7', 'XS-htROI8', 'XS-htROI9', 'M-htROI3', 'L-htROI3']

analysis_list = []
exp_config = Path('../../src/config/experiments/algonauts2021/algonauts2021_3d_resnet.yml')
cfg = combine_cfgs(
    path_cfg_data=exp_config,
    list_cfg_override=['DEBUG', debug]
)

name = exp_config.name.replace('.yml', '')

# multi-layer 1 run for each ROI
analysis = tune.run(
    tune.with_parameters(
        run_single_tune_config,
        cfg=cfg
    ),
    config={
        'DATASET.ROI': tune.grid_search(rois),
        'TRAINER.CALLBACKS.BACKBONE.DEFROST_SCORE': tune.grid_search([1.]),
    },
    local_dir=cfg.RESULTS_DIR,
    resources_per_trial={"cpu": 4, "gpu": 1},
    name=name + '_multilayer_freeze_htROI_ablation',
    verbose=1,
    resume='AUTO',
)
analysis_list.append(analysis)

2022-03-21 12:08:04,007	INFO tune.py:636 -- Total run time: 1128.95 seconds (1128.84 seconds for the tuning loop).


In [45]:
exp_config = Path('../../src/config/experiments/algonauts2021/algonauts2021_3d_resnet.yml')
cfg = combine_cfgs(
    path_cfg_data=exp_config,
    list_cfg_override=['DEBUG', debug]
)

voxel_indices_dir = Path('/home/huze/Algonauts_2021_data/voxel_indexs/')

rois = [p.name.replace('.pt', '') for p in voxel_indices_dir.glob(f'*{cfg.MODEL.BACKBONE.NAME}*')]

In [46]:
rois

['i3d_rgb_htROI5',
 'i3d_rgb_htROI2',
 'i3d_rgb_htROI6',
 'i3d_rgb_htROI4',
 'i3d_rgb_htROI3',
 'i3d_rgb_htROI1']

In [62]:
# ablation model-match

analysis_list = []
for exp_config in exp_configs[::-1]:
    # exp_config = Path('../../src/config/experiments/algonauts2021/algonauts2021_3d_resnet.yml')
    cfg = combine_cfgs(
        path_cfg_data=exp_config,
        list_cfg_override=['DEBUG', debug]
    )

    voxel_indices_dir = Path('/home/huze/Algonauts_2021_data/voxel_indexs/')

    rois = [p.name.replace('.pt', '') for p in voxel_indices_dir.glob(f'*{cfg.MODEL.BACKBONE.NAME}*')]

    name = exp_config.name.replace('.yml', '')

    # multi-layer 1 run for each ROI
    analysis = tune.run(
        tune.with_parameters(
            run_single_tune_config,
            cfg=cfg
        ),
        config={
            'DATASET.ROI': tune.grid_search(rois),
            'TRAINER.CALLBACKS.BACKBONE.DEFROST_SCORE': tune.grid_search([1.]),
        },
        local_dir=cfg.RESULTS_DIR,
        resources_per_trial={"cpu": 4, "gpu": 1},
        name=name + '_multilayer_freeze_htROI_match',
        verbose=1,
        resume='AUTO',
    )
    analysis_list.append(analysis)

Epoch 14:  55%|█████▌    | 138/250 [00:21<00:17,  6.42it/s, loss=0.181, v_num=., val_mse_loss/final=0.194, val_corr=0.0195, hp_metric=0.0236, train_mse_loss/final=0.186]



KeyboardInterrupt



In [60]:
print(rois)

['i3d_rgb_htROI5', 'i3d_rgb_htROI2', 'i3d_rgb_htROI6', 'i3d_rgb_htROI4', 'i3d_rgb_htROI3', 'i3d_rgb_htROI1']


In [26]:
ll /data/huze/ray_results/algonauts2021/

total 876
drwxrwxr-x  12 huze  4096 Mar 19 02:29 algonauts2021_2d_bdcnvgg_multilayer_defrost_anatomicalROI/
drwxrwxr-x  12 huze  4096 Mar 19 02:37 algonauts2021_2d_bdcnvgg_multilayer_freeze_anatomicalROI/
drwxrwxr-x  10 huze  4096 Mar 19 06:23 algonauts2021_2d_bdcnvgg_multilayer_freeze_htROI/
drwxrwxr-x 162 huze 36864 Mar 19 02:14 algonauts2021_2d_bdcnvgg_singlelayer_defrost_anatomicalROI/
drwxrwxr-x  12 huze  4096 Mar 18 23:12 algonauts2021_2d_colorizer_multilayer_defrost_anatomicalROI/
drwxrwxr-x  12 huze  4096 Mar 19 02:37 algonauts2021_2d_colorizer_multilayer_freeze_anatomicalROI/
drwxrwxr-x  10 huze  4096 Mar 19 07:34 algonauts2021_2d_colorizer_multilayer_freeze_htROI/
drwxrwxr-x 162 huze 45056 Mar 18 23:12 algonauts2021_2d_colorizer_singlelayer_defrost_anatomicalROI/
drwxrwxr-x  12 huze  4096 Mar 18 23:12 algonauts2021_2d_densnet_multilayer_defrost_anatomicalROI/
drwxrwxr-x  12 huze  4096 Mar 19 02:37 algonauts2021_2d_densnet_multilayer_freeze_anatomicalROI/
drwxrwxr-x  10 huze  

E0321 12:11:58.818262897  273619 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [63]:
# load result df and find backbone deforest score

In [64]:
from src.utils.runs import load_run_df, filter_multi_layer_runs, filter_single_layer_runs, my_query_df

In [65]:
run_df = load_run_df(cfg.RESULTS_DIR)

  0%|          | 0/3814 [00:00<?, ?it/s]

In [66]:
run_df = filter_multi_layer_runs(run_df)
run_df = my_query_df(run_df, equal_dict={'TRAINER.CALLBACKS.BACKBONE.DEFROST_SCORE': 1.0})

In [67]:
df = run_df[['MODEL.BACKBONE.NAME', 'DATASET.ROI', 'score']]

In [68]:
from src.utils.runs import mahou_list

In [69]:
defrost_df = df.copy()
minlestone_p = 0.5
defrost_df['defrost_score'] = defrost_df['score'].apply(lambda x: mahou_list(minlestone_p * x))

In [70]:
defrost_df

,MODEL.BACKBONE.NAME,DATASET.ROI,score,defrost_score
0,i3d_flow,i3d_flow_htROI1,0.206220,0.102
9,i3d_flow,S-htROI4,0.076862,0.039
10,i3d_flow,S-htROI6,0.174441,0.087
11,i3d_flow,S-htROI8,0.310225,0.156
12,i3d_flow,S-htROI1,0.348041,0.174
...,...,...,...,...
3801,3d_swin,3d_swin_htROI4,0.278513,0.138
3802,3d_swin,3d_swin_htROI1,0.303498,0.153
3803,3d_swin,3d_swin_htROI3,0.038273,0.018
3804,3d_swin,3d_swin_htROI2,0.229269,0.114


In [71]:
defrost_df['MODEL.BACKBONE.NAME'].unique()

array(['i3d_flow', '2d_moby_swin_warp_3d', '2d_seg_swin_warp_3d',
       'i3d_rgb', '2d_colorizer_warp_3d', '2d_densnet_warp_3d',
       '2d_simclr_warp_3d', '2d_bdcnvgg_warp_3d', '3d_swin',
       'audio_vggish', '2d_pyconvsegnet_warp_3d'], dtype=object)

In [72]:
!mkdir ./config

mkdir: cannot create directory ‘./config’: File exists


E0325 07:43:50.383993938  273619 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [73]:
defrost_df.to_csv('./config/defrost_df.csv')